# Método Triaxial — Notebook 03  
## Análisis de noticias y casos reales con matriz 3×N

Este notebook muestra cómo usar el **Método Triaxial de Discernimiento (F–C–P)**  
para evaluar noticias, narrativas o casos concretos usando la **matriz completa de 15 criterios**:

- 5 criterios de **Fundamento (F)**  
- 5 criterios de **Contexto (C)**  
- 5 criterios de **Principio (P)**  

Objetivos de este notebook:

1. Evaluar una noticia / caso **paso a paso** respondiendo los 15 criterios.  
2. Guardar los resultados en una tabla para compararlos.  
3. Dejar la base para automatizar o integrar este proceso en otros sistemas.


In [1]:
from dataclasses import dataclass
from typing import List, Dict
import pandas as pd


@dataclass
class Criterion:
    axis: str          # 'F', 'C' o 'P'
    code: str          # F1, F2, ..., P5
    name: str          # Nombre corto
    question: str      # Pregunta / criterio operativo


# 15 criterios del cuadro de evaluación
CRITERIA: List[Criterion] = [
    # FUNDAMENTO (F)
    Criterion("F", "F1", "Fuente confiable", 
              "La fuente es confiable / verificable"),
    Criterion("F", "F2", "Consistencia con hechos", 
              "La afirmación es consistente con hechos conocidos"),
    Criterion("F", "F3", "Funcionamiento real del mundo", 
              "Coincide con el funcionamiento real del mundo"),
    Criterion("F", "F4", "Sin suposiciones forzadas", 
              "No requiere suposiciones forzadas para sostenerse"),
    Criterion("F", "F5", "Sin contradicción fuerte", 
              "No contradice evidencia sólida"),

    # CONTEXTO (C)
    Criterion("C", "C1", "Contexto claro", 
              "Existe contexto claro y transparente"),
    Criterion("C", "C2", "Sin presión emocional/manipulación", 
              "No proviene de emoción, presión o manipulación"),
    Criterion("C", "C3", "Sin estigmatizar grupos", 
              "No generaliza ni estigmatiza grupos completos"),
    Criterion("C", "C4", "Sin agendas interesadas", 
              "Está aislada de agendas políticas o mediáticas interesadas"),
    Criterion("C", "C5", "Marco realista", 
              "Los datos o hechos están situados en un marco realista"),

    # PRINCIPIO (P)
    Criterion("P", "P1", "Menor entropía", 
              "Produce menor entropía (menos caos/confusión)"),
    Criterion("P", "P2", "Coherencia interna y externa", 
              "Fomenta coherencia interna y externa"),
    Criterion("P", "P3", "Respeto a la dignidad", 
              "Respeta la dignidad humana e institucional"),
    Criterion("P", "P4", "Sin odio ni división", 
              "No alimenta odio, división o violencia"),
    Criterion("P", "P5", "Acción responsable", 
              "Abre caminos de acción responsable"),
]


def criteria_template() -> pd.DataFrame:
    """Devuelve una tabla con los criterios y columna 'valor' en blanco."""
    df = pd.DataFrame([c.__dict__ for c in CRITERIA])
    df["valor"] = None
    return df


def decide_action(D: float) -> str:
    """Misma regla que en el CLI."""
    if D > 0.80:
        return "Adoptar y, si es útil, compartir"
    elif 0.50 <= D <= 0.80:
        return "Considerar válido, complementar con más análisis"
    elif 0.20 <= D < 0.50:
        return "Cuestionar, corregir o contrastar fuentes"
    else:
        return "Descartar o denunciar si es dañino / falso"


def axis_average(df: pd.DataFrame, axis: str) -> float:
    """Promedio de un eje (F, C o P) usando la columna 'valor'."""
    vals = df.loc[df["axis"] == axis, "valor"].astype(float)
    return float(vals.mean())


@dataclass
class TriaxialMatrixResult:
    claim: str
    F: float
    C: float
    P: float
    D: float
    action: str
    table: pd.DataFrame  # criterios completos


def evaluate_with_matrix(claim: str, scores: Dict[str, float]) -> TriaxialMatrixResult:
    """
    scores: diccionario { 'F1':0.3, 'F2':0.5, ..., 'P5':0.8 }
    """
    df = criteria_template()
    df["valor"] = df["code"].map(scores)

    if df["valor"].isnull().any():
        missing = df.loc[df["valor"].isnull(), "code"].tolist()
        raise ValueError(f"Faltan valores para: {missing}")

    F_val = axis_average(df, "F")
    C_val = axis_average(df, "C")
    P_val = axis_average(df, "P")

    D_val = (F_val + C_val + P_val) / 3.0
    action = decide_action(D_val)

    return TriaxialMatrixResult(
        claim=claim,
        F=F_val,
        C=C_val,
        P=P_val,
        D=D_val,
        action=action,
        table=df
    )


In [2]:
def interactive_news_evaluation() -> TriaxialMatrixResult:
    print("=== Evaluación triaxial de noticia / caso ===")
    claim = input("Resumen breve de la noticia/caso a evaluar:\n> ")

    print("\nCalifica cada criterio entre 0.0 y 1.0")
    print("Sugerencia: 1.0 = muy alto, 0.5 = dudoso/neutro, 0.0 = falso/inconsistente.\n")

    scores: Dict[str, float] = {}

    for c in CRITERIA:
        while True:
            try:
                raw = input(f"[{c.axis}] {c.code} — {c.question} (0.0–1.0): ")
                val = float(raw)
                if not (0.0 <= val <= 1.0):
                    raise ValueError
                scores[c.code] = val
                break
            except ValueError:
                print("  ⚠ Ingresa un número válido entre 0.0 y 1.0.")

    res = evaluate_with_matrix(claim, scores)

    print("\n=== Resultado Triaxial ===")
    print("Claim          :", res.claim)
    print(f"F (Fundamento) : {res.F:.2f}")
    print(f"C (Contexto)   : {res.C:.2f}")
    print(f"P (Principio)  : {res.P:.2f}")
    print(f"D (Discernim.) : {res.D:.2f}")
    print("Acción sugerida:", res.action)

    return res


In [3]:
resultado_1 = interactive_news_evaluation()


=== Evaluación triaxial de noticia / caso ===


Resumen breve de la noticia/caso a evaluar:
>  Gobierno anuncia nueva reforma educativa para mejorar salarios docentes



Califica cada criterio entre 0.0 y 1.0
Sugerencia: 1.0 = muy alto, 0.5 = dudoso/neutro, 0.0 = falso/inconsistente.



[F] F1 — La fuente es confiable / verificable (0.0–1.0):  0.8
[F] F2 — La afirmación es consistente con hechos conocidos (0.0–1.0):  0.8
[F] F3 — Coincide con el funcionamiento real del mundo (0.0–1.0):  0.8
[F] F4 — No requiere suposiciones forzadas para sostenerse (0.0–1.0):  0.8
[F] F5 — No contradice evidencia sólida (0.0–1.0):  0.8
[C] C1 — Existe contexto claro y transparente (0.0–1.0):  0.8
[C] C2 — No proviene de emoción, presión o manipulación (0.0–1.0):  0.8
[C] C3 — No generaliza ni estigmatiza grupos completos (0.0–1.0):  0.8
[C] C4 — Está aislada de agendas políticas o mediáticas interesadas (0.0–1.0):  0.5
[C] C5 — Los datos o hechos están situados en un marco realista (0.0–1.0):  0.8
[P] P1 — Produce menor entropía (menos caos/confusión) (0.0–1.0):  0.8
[P] P2 — Fomenta coherencia interna y externa (0.0–1.0):  0.8
[P] P3 — Respeta la dignidad humana e institucional (0.0–1.0):  0.8
[P] P4 — No alimenta odio, división o violencia (0.0–1.0):  0.8
[P] P5 — Abre caminos de ac


=== Resultado Triaxial ===
Claim          : Gobierno anuncia nueva reforma educativa para mejorar salarios docentes
F (Fundamento) : 0.80
C (Contexto)   : 0.74
P (Principio)  : 0.80
D (Discernim.) : 0.78
Acción sugerida: Considerar válido, complementar con más análisis


In [4]:
resultado_1.table


,axis,code,name,question,valor
0,F,F1,Fuente confiable,La fuente es confiable / verificable,0.8
1,F,F2,Consistencia con hechos,La afirmación es consistente con hechos conocidos,0.8
2,F,F3,Funcionamiento real del mundo,Coincide con el funcionamiento real del mundo,0.8
3,F,F4,Sin suposiciones forzadas,No requiere suposiciones forzadas para sostenerse,0.8
4,F,F5,Sin contradicción fuerte,No contradice evidencia sólida,0.8
5,C,C1,Contexto claro,Existe contexto claro y transparente,0.8
6,C,C2,Sin presión emocional/manipulación,"No proviene de emoción, presión o manipulación",0.8
7,C,C3,Sin estigmatizar grupos,No generaliza ni estigmatiza grupos completos,0.8
8,C,C4,Sin agendas interesadas,Está aislada de agendas políticas o mediáticas...,0.5
9,C,C5,Marco realista,Los datos o hechos están situados en un marco ...,0.8


In [5]:
results_list: List[TriaxialMatrixResult] = []

# Añadimos el primer resultado si ya existe
results_list.append(resultado_1)


def results_summary(results: List[TriaxialMatrixResult]) -> pd.DataFrame:
    """Tabla resumen por noticia/caso."""
    data = [
        {
            "Claim": r.claim,
            "F (Fundamento)": round(r.F, 2),
            "C (Contexto)": round(r.C, 2),
            "P (Principio)": round(r.P, 2),
            "D (Discernimiento)": round(r.D, 2),
            "Acción sugerida": r.action,
        }
        for r in results
    ]
    return pd.DataFrame(data)


In [6]:
print("Vamos a evaluar otra noticia/caso.\n")
resultado_2 = interactive_news_evaluation()
results_list.append(resultado_2)

summary_df = results_summary(results_list)
summary_df


Vamos a evaluar otra noticia/caso.

=== Evaluación triaxial de noticia / caso ===


Resumen breve de la noticia/caso a evaluar:
>  El gobierno esta dirigido por el narcotrafico



Califica cada criterio entre 0.0 y 1.0
Sugerencia: 1.0 = muy alto, 0.5 = dudoso/neutro, 0.0 = falso/inconsistente.



[F] F1 — La fuente es confiable / verificable (0.0–1.0):  0.3
[F] F2 — La afirmación es consistente con hechos conocidos (0.0–1.0):  0.5
[F] F3 — Coincide con el funcionamiento real del mundo (0.0–1.0):  0.5
[F] F4 — No requiere suposiciones forzadas para sostenerse (0.0–1.0):  0.2
[F] F5 — No contradice evidencia sólida (0.0–1.0):  0.2
[C] C1 — Existe contexto claro y transparente (0.0–1.0):  0.2
[C] C2 — No proviene de emoción, presión o manipulación (0.0–1.0):  0.2
[C] C3 — No generaliza ni estigmatiza grupos completos (0.0–1.0):  0.2
[C] C4 — Está aislada de agendas políticas o mediáticas interesadas (0.0–1.0):  0.2
[C] C5 — Los datos o hechos están situados en un marco realista (0.0–1.0):  0.5
[P] P1 — Produce menor entropía (menos caos/confusión) (0.0–1.0):  0.2
[P] P2 — Fomenta coherencia interna y externa (0.0–1.0):  0.2
[P] P3 — Respeta la dignidad humana e institucional (0.0–1.0):  0.2
[P] P4 — No alimenta odio, división o violencia (0.0–1.0):  0.2
[P] P5 — Abre caminos de ac


=== Resultado Triaxial ===
Claim          : El gobierno esta dirigido por el narcotrafico
F (Fundamento) : 0.34
C (Contexto)   : 0.26
P (Principio)  : 0.20
D (Discernim.) : 0.27
Acción sugerida: Cuestionar, corregir o contrastar fuentes


,Claim,F (Fundamento),C (Contexto),P (Principio),D (Discernimiento),Acción sugerida
0,Gobierno anuncia nueva reforma educativa para ...,0.80,0.74,0.8,0.78,"Considerar válido, complementar con más análisis"
1,El gobierno esta dirigido por el narcotrafico,0.34,0.26,0.2,0.27,"Cuestionar, corregir o contrastar fuentes"


In [7]:
summary_df.set_index("Claim")[["F (Fundamento)", "C (Contexto)", "P (Principio)"]]


,F (Fundamento),C (Contexto),P (Principio)
Claim,,,
Gobierno anuncia nueva reforma educativa para mejorar salarios docentes,0.80,0.74,0.8
El gobierno esta dirigido por el narcotrafico,0.34,0.26,0.2
